# Modelos de precio

In [1]:
import QuantLib as ql

En un modulo anterior vimos el Engine de BlackScholes, analicemos otros ademas. Montemos el framework en QuantLib primero

In [2]:
precio_activo = 440 #Spot (S)
precio_ejercicio = 450 #Strike (K)
volatilidad = 0.3394  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.0013 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.075 #tasa de dividendos anualizada (q)

In [3]:
#Expiracion el 16 de octubre de 2020
fecha_expiracion = ql.Date(16, 10, 2020)

#Valuacion el 16 de octubre de 2020
fecha_valuacion = ql.Date(6, 8, 2020)
ql.Settings.instance().evaluationDate = fecha_valuacion

#Calendario y convencion de daycount
day_count = ql.Actual365Fixed()
calendario = ql.UnitedStates()

In [4]:
tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)
ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)
opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)

In [5]:
S = ql.SimpleQuote(precio_activo)
objeto_spot = ql.QuoteHandle(S)

r = ql.SimpleQuote(tasa_interes)
objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                            ql.QuoteHandle(r), 
                                                        day_count))
q = ql.SimpleQuote(tasa_dividendos)
objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      ql.QuoteHandle(q), 
                                                      day_count))

sigma = ql.SimpleQuote(volatilidad)
objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 ql.QuoteHandle(sigma), 
                                                             day_count))

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)

Ahora que tenemos el proceso armado llego finalmente la hora de elegir un modelo para poner precio y hallarlo. 

Para esto necesitamos 3 ingredientes:

1) La opcion (en nuestro ejemplo `opcion_europea`)

2) El proceso (en nuestro ejemplo `proceso_BSM`)

3) El modelo (Engine)

Al objeto opcion que es parte de `instrument`, le aplico el metodo `setPricingEngine`

Quantlib ofrece una libreria muy extensa de modelos de pricing para Opciones. En este tipo de cosas reside el poder de QuantLib. Estos son los modelos existentes en QuantLib

`VarianceGammaEngine`, `FFTEngine`, `FFTVanillaEngine`, `FFTVarianceGammaEngine`, `AnalyticBSMHullWhiteEngine`, `AnalyticDigitalAmericanEngine`, `AnalyticDigitalAmericanKOEngine`, `AnalyticDividendEuropeanEngine`, `AnalyticEuropeanEngine`, `AnalyticH1HWEngine`, `AnalyticHestonEngine`, `AnalyticHestonHullWhiteEngine`, `AnalyticPTDHestonEngine`, `BaroneAdesiWhaleyApproximationEngine`, `BatesEngine`, `BinomialVanillaEngine`, `BjerksundStenslandApproximationEngine`, `COSHestonEngine`, `FdBatesVanillaEngine`, `FDBermudanEngine`, `FDDividendEngineMerton73`, `FDDividendEngineShiftScale`, `FDDividendShoutEngine`, `FdHestonHullWhiteVanillaEngine`, `FDShoutEngine`, `FDStepConditionEngine`, `FDVanillaEngine`, `HestonExpansionEngine`, `IntegralEngine`, `JumpDiffusionEngine`, `JuQuadraticApproximationEngine`, `MCAmericanEngine`, `MCDigitalEngine`, `MCEuropeanEngine`, `MCEuropeanGJRGARCHEngine`, `MCEuropeanHestonEngine`, `MCVanillaEngine`,


Veamos cuatro de estos modelos, que suelen ser los primeros que uno ve cuando estudia valuacion de opciones:
* Analitico - Formula cerrada, Black Scholes propiamente dicho
* Diferencias Finitas - Resolucion de la ecuacion diferencial de Black Sholes
* Montecarlo - Simulaciones del proceso de BSM y promedio descontado
* Binomial - Arbol Binomial de un proceso que en el limite tiende al proceso de BSM

La clase `VanillaOption` tiene los siguientes posibles salidas: `NPV()` - este nos dara el Net  Present Value del contrato; `errorEstimate()` - este nos dara informacion de estimacion si usamos un modelo de aproximacion.

## Modelo (Black Scholes) [`AnalyticEuropeanEngine`]

In [6]:
modelo_BS = ql.AnalyticEuropeanEngine(proceso_BSM)

opcion_europea.setPricingEngine(modelo_BS)

precio_opcion_BS = opcion_europea.NPV()

print("El precio teorico usando el modelo de BS es: ", precio_opcion_BS)

El precio teorico usando el modelo de BS es:  19.018046859717114


En el caso de la formula Analitica, QuantLib ofrece la posibilidad de extraer varias sensibilidades (griegas) de la opcion. Entre ellos: `delta()`,  `vega()`, `rho() `, `dividendRho()`, `theta()`, `thetaPerDay()`, `strikeSensitivity()`, `gamma()`, `elasticity()`.


In [7]:
delta_BS = opcion_europea.delta()
vega_BS = opcion_europea.vega()
rho_BS = opcion_europea.rho() 
div_rho_BS = opcion_europea.dividendRho()
theta_BS = opcion_europea.theta()
thetaPD_BS = opcion_europea.thetaPerDay()
strike_greek_BS = opcion_europea.strikeSensitivity()

gamma_BS = opcion_europea.gamma()
elasticity_BS = opcion_europea.elasticity()

Aqui mostramos las griegas obtenidas

In [8]:
print("Delta: ", delta_BS)
print("Vega:", vega_BS)
print("Rho: ", rho_BS)
print("dividendRho: ", div_rho_BS)
print("Theta: ", theta_BS)
print("Theta_PD: ", thetaPD_BS)
print("strikeSensitivity: ", strike_greek_BS)

print("Gamma: ", gamma_BS)
print("Elasticity", elasticity_BS)

Delta:  0.4258321048466277
Vega: 75.18926359141635
Rho:  32.74716062566774
dividendRho:  -36.446561521667505
Theta:  -51.76161250063861
Theta_PD:  -0.141812636988051
strikeSensitivity:  -0.374106842828442
Gamma:  0.0058826519800777724
Elasticity 9.852017271520342


## Modelo de  Diferencias Finitas [ `FdBlackScholesVanillaEngine` ]

Al Engine de Diferencias Finitas se le puede indicar el esquema de diferencias finitas a utilizar en el input `schemeDesc`:

* `Hundsdorfer`
* `Douglas`
* `CraigSneyd`
* `ModifiedCraigSneyd`
* `ImplicitEuler`
* `ExplicitEuler`
* `MethodOfLines`
 
Tambine se puede determinar el tamaño de la grilla con `tGrid` y `xGrid` y `dampingSteps`

In [9]:
tGrid = 100
xGrid = 100
dampingSteps = 0
esquema_df = ql.FdmSchemeDesc.MethodOfLines()

modelo_DF = ql.FdBlackScholesVanillaEngine(proceso_BSM, 
                                           tGrid, 
                                           xGrid,
                                           dampingSteps,
                                           esquema_df)

opcion_europea.setPricingEngine(modelo_DF)

precio_opcion_DF = opcion_europea.NPV()

print("El precio teorico usando el modelo de Dif. Finitas es: ", precio_opcion_DF)

El precio teorico usando el modelo de Dif. Finitas es:  19.0258645465868


En el caso del modelo de Diferencias Finitas QuantLib ofrece la posibilidad de extraer tres greigas: `delta()`,  `theta()`, `gamma()`.

In [10]:

delta_DF = opcion_europea.delta()
theta_DF = opcion_europea.theta()

gamma_DF = opcion_europea.gamma()

print("Delta: ", delta_DF)
print("Theta: ", theta_DF)

print("Gamma: ", gamma_DF)



Delta:  0.4261266324203341
Theta:  -52.02236667561396
Gamma:  0.005887125208638206


## Modelo de Montecarlo [ `MCEuropeanEngine` ]



El Engine de MonteCarlo pide como entrada:
* Forma de generacion de numeros aleaorios [necesario]
* paso del tiempo (timeSteps) o pasos de tiempo por año (timeStepsPerYear)
* cantidad de caminos (requiredSamples)
* tolerancia (requiredTolerance)
* maximo de caminos (maxSamples)
* semilla de generador de numeros aleatorios (seed)




In [11]:
generador_numeros_aleatorios = "PseudoRandom" 

#generador_numeros_aleatorios = "MersenneTwisterUniformRng"

pasos_tiempo=20
caminos = 500000
modelo_MC = ql.MCEuropeanEngine(proceso_BSM, 
                                generador_numeros_aleatorios, 
                                timeSteps = pasos_tiempo,
                                requiredSamples = caminos)

opcion_europea.setPricingEngine(modelo_MC)

precio_opcion_MC = opcion_europea.NPV()

print("El precio teorico usando el modelo de MonteCarlo es: ",  precio_opcion_MC)

El precio teorico usando el modelo de MonteCarlo es:  19.05382395826194


Como es sabido, un gran deficit del metodo de MonteCarlo es la obtencion de griegas. En este caso QuantLib no provee metodos de obtencion de griegas

Lo que si ofrece como salida es el metodo `errorEstimate()`, que describe el error del metodo de Montecarlo.



In [12]:
error_MC = opcion_europea.errorEstimate()
print("Error del metodo de MonteCarlo: ", error_MC)


Error del metodo de MonteCarlo:  0.051012461987428974


## Modelo del Arbol Binomial [ `BinomialVanillaEngine`]

El Engine Binomial pide como entrada un modelo de arbol. QuantLib soporta, entre otros los siguientes:

* CRR / CoxRossRubinstein

* JOSHI / Joshi4

* JR / JarrowRudd 

* LR / LeisenReimer

* TIAN / Tian 

* TRI / Trigeorgis 

Por ultimo, pide como entrada la cantidad de pasos del arbol como tercer parametro.


In [13]:
#por ejemplo, usamos el modelo de Leisner Reimer

modelo_arbol = 'LR'
cant_pasos_arbol = 1000

modelo_Bin = ql.BinomialVanillaEngine(proceso_BSM, 
                                      modelo_arbol, 
                                      cant_pasos_arbol)

opcion_europea.setPricingEngine(modelo_Bin)

precio_opcion_Bin = opcion_europea.NPV()

print("El precio teorico usando el modelo Binomial es: ",  precio_opcion_Bin)

El precio teorico usando el modelo Binomial es:  18.99545340064186


En el caso del modelo Binomial, al igual que el el caso de Diferencias finitas, QuantLib ofrece la posibilidad de extraer tres greigas: `delta()`,  `theta()`, `gamma()`.

In [14]:
delta_Bin = opcion_europea.delta()
theta_Bin = opcion_europea.theta()

gamma_Bin = opcion_europea.gamma()


print("Delta: ", delta_Bin)
print("Theta: ", theta_Bin)
print("Gamma: ", gamma_Bin)


Delta:  0.4258059821803505
Theta:  -51.858632977833054
Gamma:  0.0058912742805174645


## Black - Vasicek - tests


In [36]:

method = "Heston semi-analytic"

precio_activo = 440 #Spot (S)
precio_ejercicio = 450 #Strike (K)
volatilidad = 0.3394  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.0013 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.075 #tasa de dividendos anualizada (q)


#riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, ql.Actual365Fixed()))
#dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.01, ql.Actual365Fixed()))

initialValue = ql.QuoteHandle(ql.SimpleQuote(100))
v0 = 0.05
kappa = 10
theta = 0.08
rho = 0.51

hestonProcess = ql.HestonProcess(objeto_tasa_interes, 
                                 objeto_tasa_dividendos, 
                                 objeto_spot, 
                                 v0, 
                                 kappa, 
                                 theta, 
                                 volatilidad, 
                                 rho)

hestonModel = ql.HestonModel(hestonProcess)


modelo_heston = ql.AnalyticHestonEngine(hestonModel)




opcion_europea.setPricingEngine(modelo_heston)
    
print("Heston: ", opcion_europea.NPV())

Heston:  13.201860317929999


## Extracto C++ para Heston

In [ ]:


        std::vector<Date> exerciseDates;
        for (Integer i=1; i<=4; i++)
            exerciseDates.push_back(settlementDate + 3*i*Months);

        ext::shared_ptr<Exercise> europeanExercise(
                                         new EuropeanExercise(maturity));

        ext::shared_ptr<Exercise> bermudanExercise(
                                         new BermudanExercise(exerciseDates));

        ext::shared_ptr<Exercise> americanExercise(
                                         new AmericanExercise(settlementDate,
                                                              maturity));

        Handle<Quote> underlyingH(
            ext::shared_ptr<Quote>(new SimpleQuote(underlying)));

        // bootstrap the yield/dividend/vol curves
        Handle<YieldTermStructure> flatTermStructure(
            ext::shared_ptr<YieldTermStructure>(
                new FlatForward(settlementDate, riskFreeRate, dayCounter)));
        Handle<YieldTermStructure> flatDividendTS(
            ext::shared_ptr<YieldTermStructure>(
                new FlatForward(settlementDate, dividendYield, dayCounter)));
        Handle<BlackVolTermStructure> flatVolTS(
            ext::shared_ptr<BlackVolTermStructure>(
                new BlackConstantVol(settlementDate, calendar, volatility,
                                     dayCounter)));
        ext::shared_ptr<StrikedTypePayoff> payoff(
                                        new PlainVanillaPayoff(type, strike));
        ext::shared_ptr<BlackScholesMertonProcess> bsmProcess(
                 new BlackScholesMertonProcess(underlyingH, flatDividendTS,
                                               flatTermStructure, flatVolTS));

        // options
        VanillaOption europeanOption(payoff, europeanExercise);
        VanillaOption bermudanOption(payoff, bermudanExercise);
        VanillaOption americanOption(payoff, americanExercise);

        // Analytic formulas:

        // Black-Scholes for European
        method = "Black-Scholes";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                                     new AnalyticEuropeanEngine(bsmProcess)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        //Vasicek rates model for European
        method = "Black Vasicek Model";
        Real r0 = riskFreeRate;
        Real a = 0.3;
        Real b = 0.3;
        Real sigma_r = 0.15;
        Real riskPremium = 0.0;
        Real correlation = 0.5;
        ext::shared_ptr<Vasicek> vasicekProcess(new Vasicek(r0, a, b, sigma_r, riskPremium));
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new AnalyticBlackVasicekEngine(bsmProcess, vasicekProcess, correlation)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // semi-analytic Heston for European
        method = "Heston semi-analytic";
        ext::shared_ptr<HestonProcess> hestonProcess(
            new HestonProcess(flatTermStructure, flatDividendTS,
                              underlyingH, volatility*volatility,
                              1.0, volatility*volatility, 0.001, 0.0));
        ext::shared_ptr<HestonModel> hestonModel(
                                              new HestonModel(hestonProcess));
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                                     new AnalyticHestonEngine(hestonModel)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // semi-analytic Bates for European
        method = "Bates semi-analytic";
        ext::shared_ptr<BatesProcess> batesProcess(
            new BatesProcess(flatTermStructure, flatDividendTS,
                             underlyingH, volatility*volatility,
                             1.0, volatility*volatility, 0.001, 0.0,
                             1e-14, 1e-14, 1e-14));
        ext::shared_ptr<BatesModel> batesModel(new BatesModel(batesProcess));
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                                                new BatesEngine(batesModel)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // Barone-Adesi and Whaley approximation for American
        method = "Barone-Adesi/Whaley";
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                       new BaroneAdesiWhaleyApproximationEngine(bsmProcess)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << "N/A"
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Bjerksund and Stensland approximation for American
        method = "Bjerksund/Stensland";
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BjerksundStenslandApproximationEngine(bsmProcess)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << "N/A"
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Integral
        method = "Integral";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                                             new IntegralEngine(bsmProcess)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // Finite differences
        Size timeSteps = 801;
        method = "Finite differences";
        ext::shared_ptr<PricingEngine> fdengine =
            ext::make_shared<FdBlackScholesVanillaEngine>(bsmProcess,
                                                          timeSteps,
                                                          timeSteps-1);
        europeanOption.setPricingEngine(fdengine);
        bermudanOption.setPricingEngine(fdengine);
        americanOption.setPricingEngine(fdengine);
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Jarrow-Rudd
        method = "Binomial Jarrow-Rudd";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<JarrowRudd>(bsmProcess,timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<JarrowRudd>(bsmProcess,timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<JarrowRudd>(bsmProcess,timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;
        method = "Binomial Cox-Ross-Rubinstein";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<CoxRossRubinstein>(bsmProcess,
                                                                   timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<CoxRossRubinstein>(bsmProcess,
                                                                   timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<CoxRossRubinstein>(bsmProcess,
                                                                   timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Additive equiprobabilities
        method = "Additive equiprobabilities";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<AdditiveEQPBinomialTree>(bsmProcess,
                                                                   timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<AdditiveEQPBinomialTree>(bsmProcess,
                                                                   timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<AdditiveEQPBinomialTree>(bsmProcess,
                                                                   timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Binomial Trigeorgis
        method = "Binomial Trigeorgis";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<Trigeorgis>(bsmProcess,timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<Trigeorgis>(bsmProcess,timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                new BinomialVanillaEngine<Trigeorgis>(bsmProcess,timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Binomial Tian
        method = "Binomial Tian";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<Tian>(bsmProcess,timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<Tian>(bsmProcess,timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                      new BinomialVanillaEngine<Tian>(bsmProcess,timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Binomial Leisen-Reimer
        method = "Binomial Leisen-Reimer";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
              new BinomialVanillaEngine<LeisenReimer>(bsmProcess,timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
              new BinomialVanillaEngine<LeisenReimer>(bsmProcess,timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
              new BinomialVanillaEngine<LeisenReimer>(bsmProcess,timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Binomial method: Binomial Joshi
        method = "Binomial Joshi";
        europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                    new BinomialVanillaEngine<Joshi4>(bsmProcess,timeSteps)));
        bermudanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                    new BinomialVanillaEngine<Joshi4>(bsmProcess,timeSteps)));
        americanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
                    new BinomialVanillaEngine<Joshi4>(bsmProcess,timeSteps)));
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << bermudanOption.NPV()
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // Monte Carlo Method: MC (crude)
        timeSteps = 1;
        method = "MC (crude)";
        Size mcSeed = 42;
        ext::shared_ptr<PricingEngine> mcengine1;
        mcengine1 = MakeMCEuropeanEngine<PseudoRandom>(bsmProcess)
            .withSteps(timeSteps)
            .withAbsoluteTolerance(0.02)
            .withSeed(mcSeed);
        europeanOption.setPricingEngine(mcengine1);
        // Real errorEstimate = europeanOption.errorEstimate();
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // Monte Carlo Method: QMC (Sobol)
        method = "QMC (Sobol)";
        Size nSamples = 32768;  // 2^15

        ext::shared_ptr<PricingEngine> mcengine2;
        mcengine2 = MakeMCEuropeanEngine<LowDiscrepancy>(bsmProcess)
            .withSteps(timeSteps)
            .withSamples(nSamples);
        europeanOption.setPricingEngine(mcengine2);
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << europeanOption.NPV()
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << "N/A"
                  << std::endl;

        // Monte Carlo Method: MC (Longstaff Schwartz)
        method = "MC (Longstaff Schwartz)";
        ext::shared_ptr<PricingEngine> mcengine3;
        mcengine3 = MakeMCAmericanEngine<PseudoRandom>(bsmProcess)
            .withSteps(100)
            .withAntitheticVariate()
            .withCalibrationSamples(4096)
            .withAbsoluteTolerance(0.02)
            .withSeed(mcSeed);
        americanOption.setPricingEngine(mcengine3);
        std::cout << std::setw(widths[0]) << std::left << method
                  << std::fixed
                  << std::setw(widths[1]) << std::left << "N/A"
                  << std::setw(widths[2]) << std::left << "N/A"
                  << std::setw(widths[3]) << std::left << americanOption.NPV()
                  << std::endl;

        // End test
        return 0;

    } catch (std::exception& e) {
        std::cerr << e.what() << std::endl;
        return 1;
    } catch (...) {
        std::cerr << "unknown error" << std::endl;
        return 1;
    }
}
© 2020 GitHub, Inc.
Terms
Privacy
Security
Status
Help
Contact GitHub
Pricing
API
Training
Blog
About
